In [4]:
import numpy as np
import re
import psycopg2
import pandas as pd
import jieba
import jieba.analyse
import jieba.posseg as pseg

In [5]:
conn = psycopg2.connect(database="law1", user="datac1", password="datac15543", host="ci.lu.im.ntu.edu.tw", port="5432")
print ("Opened database successfully")

cur = conn.cursor()

cur.execute("""SELECT sqlid,main FROM jdata_features WHERE sqlid IN (SELECT sqlid FROM jdata_meta WHERE cat = 'M' and parser_done = '1') limit 50""")
rows = cur.fetchall()

Opened database successfully


In [6]:
sample_df = pd.DataFrame(rows)
sample_df.head(50)

,0,1
0,5469352,上訴駁回。
1,5470041,原判決撤銷。丙○○犯傷害罪，處拘役拾日，如易科罰金，以新臺幣壹仟元折算壹日。
2,5470145,原判決關於甲○○加重強盜未遂及乙○○寄藏可發射子彈具有殺傷力之改造手槍部分，暨定應執行刑部分...
3,5470146,原判決撤銷。甲○○意圖供行使之用，而偽造幣券，處有期徒刑年。扣案偽造之壹仟元鈔貳拾陸張、伍...
4,5470147,原判決關於己○○部分、丙○○偽造有證券及定執行刑部分均撤銷。己○○共同犯偽造有價證券罪，處有...
5,5470148,上訴駁回。
6,5470149,上訴駁回。
7,5470150,原判決關於有罪部分撤銷。戊○○所犯如附表所示販賣第一級毒品之罪，均累犯，各處如附表所示之刑（...
8,5470152,原判決關於丙○○部分，及丁○○販賣第二級毒品部分，均撤銷。丙○○犯如附表所示之罪，處如附表所...
9,5470260,上訴駁回。


In [8]:
test_str = sample_df[1][1]
print(test_str)
print(test_str[:3])

原判決撤銷。丙○○犯傷害罪，處拘役拾日，如易科罰金，以新臺幣壹仟元折算壹日。
原判決


In [378]:
sentences = re.split('。',test_str)
sentences = [t for t in sentences if t != "" ]
print(sentences)

['辛○○公司負責人為納稅義務人，以不正當之方法逃漏稅捐，處有期徒刑拾月；又商業負責人以明知不實事項而填製會計憑證，處有期徒刑捌月；應執行有期徒刑壹年肆月', '壬○○公司負責人為納稅義務人，以不正當之方法逃漏稅捐，處有期徒刑陸月，如易科罰金，以參佰元折算壹日；又共同商業負責人以明知不實事項而填製會計憑證，處有期徒刑伍月，如易科罰金，以參佰元折算壹日；應執行有期徒刑拾月，如易科罰金，以參佰元折算壹日', '丙○○無罪']


In [379]:
no_guilty_words = ["無罪","駁回","撤銷","不受理","免訴"]
guilty_words = ["月","年","元","天","勞務","徒刑","褫奪公權"]

In [380]:
guilty_sens = []
sens_flag = 0
for sens in sentences :
    sens_flag = 0
    for w in guilty_words :
        if (sens.find(w) >= 0 and sens_flag == 0) :
            guilty_sens.append(sens)
            sens_flag = 1
print(guilty_sens)

['辛○○公司負責人為納稅義務人，以不正當之方法逃漏稅捐，處有期徒刑拾月；又商業負責人以明知不實事項而填製會計憑證，處有期徒刑捌月；應執行有期徒刑壹年肆月', '壬○○公司負責人為納稅義務人，以不正當之方法逃漏稅捐，處有期徒刑陸月，如易科罰金，以參佰元折算壹日；又共同商業負責人以明知不實事項而填製會計憑證，處有期徒刑伍月，如易科罰金，以參佰元折算壹日；應執行有期徒刑拾月，如易科罰金，以參佰元折算壹日']


In [381]:
laws_words_pattern = "從事.+|無故.+|未.+|以.+之事|連續.+|意圖.+|行使.+|犯.+[罪|規定]|違反.+[罪|規定]|依.+[法|條例|罪|規定]|販賣第[一|二|三|四、]{+}級毒品"
judgements_words_pattern = "處.+|應.+|緩刑.+"
total_words_pattern = "均.+"
add_law_pattern = "又.+"
final_judgement_pattern = "應執行.+"

In [382]:
guilty_pairs = []
law_str = ""
for sens in guilty_sens:
    last_law = law_str
    law_str = ""
    judgement_strs = []
    total_flag = 0
    add_flag = 0
    sens_semicolon = re.split('；|，',sens)
    last_sen = 0
    for semi_sens in sens_semicolon :
#         print(semi_sens)
        if (len(re.findall(laws_words_pattern,semi_sens)) > 0) :
            total_flag = 0
            add_flag = 0
            last_sen = 1
            law_str = re.findall(laws_words_pattern,semi_sens)[0]
        elif (len(re.findall(judgements_words_pattern,semi_sens)) > 0):
            total_flag = 0
            add_flag = 0
            last_sen = 2
            count = len(re.findall("、",semi_sens))
            judgement_strs = judgement_strs + [re.findall(judgements_words_pattern,semi_sens)[0] for i in range(0,count+1)]
        elif (len(re.findall(total_words_pattern,semi_sens)) > 0):
            total_flag = 1
            add_flag = 0
            last_sen = 2
            judgement_strs = [ t + "," + re.findall(total_words_pattern,semi_sens)[0] for t in judgement_strs]
        elif (len(re.findall(add_law_pattern,semi_sens)) > 0):
            total_flag = 0
            add_flag = 1
            last_sen = 1
            if law_str != "" :
                law_str = law_str + "," + semi_sens
            else :
                law_str = semi_sens[1:]
        else :
            if total_flag == 1 :
                judgement_strs = [ t + "," + semi_sens for t in judgement_strs]
            elif add_flag == 1 :
                law_str = law_str + "," + semi_sens
            else :
                if last_sen == 2 :
                    if len(judgement_strs) > 0 :
                        judgement_strs[-1] = judgement_strs[-1] + "," + semi_sens
                elif last_sen == 1 :
                    if law_str != "" :
                        law_str = law_str + "," + semi_sens
                    else :
                        law_str = semi_sens[1:]
                    
    for j_str in judgement_strs :
        if (law_str == "") :
            law_str = last_law
        guilty_pairs.append((law_str,j_str))

guilty_pairs = [t for t in guilty_pairs if t != "" ]
print(guilty_pairs)

[('商業負責人以明知不實事項而填製會計憑證', '處有期徒刑拾月'), ('商業負責人以明知不實事項而填製會計憑證', '處有期徒刑捌月'), ('商業負責人以明知不實事項而填製會計憑證', '應執行有期徒刑壹年肆月'), ('共同商業負責人以明知不實事項而填製會計憑證', '處有期徒刑陸月,如易科罰金,以參佰元折算壹日'), ('共同商業負責人以明知不實事項而填製會計憑證', '處有期徒刑伍月,如易科罰金,以參佰元折算壹日'), ('共同商業負責人以明知不實事項而填製會計憑證', '應執行有期徒刑拾月,如易科罰金,以參佰元折算壹日')]


In [383]:
no_guilty_sens = []
sens_flag = 0
for sens in sentences :
    sens_flag = 0
    for w in no_guilty_words :
        if (sens.find(w) >= 0 and sens_flag == 0) :
            no_guilty_sens.append(sens)
            sens_flag = 1
print(no_guilty_sens)

['丙○○無罪']


In [384]:
laws_words_pattern = "以.+之事|連續.+|意圖.+|行使.+|犯.+[罪|規定]|違反.+[罪|規定]|依.+[法|條例|罪|規定]|販賣第[一|二|三|四、]{+}級毒品"
judgements_words_pattern = "無罪|駁回|撤銷|不受理|免訴"
add_law_pattern = "又.+"

In [385]:
no_guilty_pairs = []
law_str = ""
for sens in no_guilty_sens:
    last_law = law_str
    law_str = ""
    judgement_strs = []
    add_flag = 0
    sens_semicolon = re.split('；|，',sens)
    last_sen = 0
    for semi_sens in sens_semicolon :
#         print(semi_sens)
        if (len(re.findall(laws_words_pattern,semi_sens)) > 0) :
            add_flag = 0
            last_sen = 1
            law_str = re.findall(laws_words_pattern,semi_sens)[0]
        elif (len(re.findall(judgements_words_pattern,semi_sens)) > 0):
            add_flag = 0
            last_sen = 2
            count = len(re.findall("、",semi_sens))
            judgement_strs = judgement_strs + [re.findall(judgements_words_pattern,semi_sens)[0] for i in range(0,count+1)]
        elif (len(re.findall(add_law_pattern,semi_sens)) > 0):
            add_flag = 1
            last_sen = 1
            if law_str != "" :
                law_str = law_str + "," + semi_sens
            else :
                law_str = semi_sens[1:]
        else :
            if add_flag == 1 :
                law_str = law_str + "," + semi_sens
            else :
                if last_sen == 2 :
                    if len(judgement_strs) > 0 :
                        judgement_strs[-1] = judgement_strs[-1] + "," + semi_sens
                elif last_sen == 1 :
                    if law_str != "" :
                        law_str = law_str + "," + semi_sens
                    else :
                        law_str = semi_sens[1:]
                    
    for j_str in judgement_strs :
        if (law_str == "") :
            law_str = last_law
        no_guilty_pairs.append((law_str,j_str))

no_guilty_pairs = [t for t in no_guilty_pairs if t != "" ]
print(no_guilty_pairs)

[('', '無罪')]
